# Import Libraries and Load Data of Links

In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Load the Excel file
data = pd.read_excel('filtered_aiims_list.xlsx') # Path to the Excel file. If in the same directory as code, provide file name.
data  # data obtained From various Website

,Unnamed: 0,name of organization,state,tender title,document link,publish date,Unnamed: 5
0,0,AIIMS BHOPAL,MADHYA PRADESH,qiastst dx system multiplexing platform syndro...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,29-Aug-2024,NaN
1,1,AIIMS BHOPAL,MADHYA PRADESH,jaypee digital medical e database central libr...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,18-Jul-2024,NaN
2,2,AIIMS BHOPAL,MADHYA PRADESH,proposal neuroblastoma cell line sh sysy pro...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,03-May-2024,NaN
3,3,AIIMS BHOPAL,MADHYA PRADESH,fish microscope software project funded dhr ic...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,06-Apr-2024,NaN
4,4,AIIMS BHOPAL,MADHYA PRADESH,autonomous human lung simulator adult pediatri...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,05-Apr-2024,NaN
...,...,...,...,...,...,...,...
2527,2527,AIIMS DELHI,DELHI,integrated difficult airway scope system adult...,https://www.aiims.edu/images/pdf/tender/Integr...,19/09/2024,NaN
2528,2528,AIIMS DELHI,DELHI,radio frequency ablation system 01no departmen...,https://www.aiims.edu/images/pdf/tender/Ablati...,19/09/2024,NaN
2529,2529,AIIMS DELHI,DELHI,teg thrombelastograph analyzer machine departm...,https://www.aiims.edu/images/pdf/tender/TEG Th...,21/09/2024,NaN
2530,2530,AIIMS DELHI,DELHI,continuous renal replacement therapy crrt depa...,https://www.aiims.edu/images/pdf/tender/CRRT.pdf,21/09/2024,NaN


## Inspect collection of links state wise

In [2]:
# Perform the groupby and value_counts operation
info = data.groupby('state').size()  # Changed to size() for a simpler view of counts

# Display the DataFrame
print(info)

state
ANDHRA PRADESH       63
BIHAR                91
CHHATTISGARH        444
DELHI               588
GUJRAT                5
HIMACHAL PRADESH     28
JHARKHAND             6
MADHYA PRADESH      178
MAHARASTRA           31
ODHISHA              31
PUNJAB               19
RAJASTHAN           523
TELANGANA            20
UTTAR PRADESH        85
UTTRAKHAND          408
WEST BENGAL          12
dtype: int64


## Inspect Links based on Years

In [2]:
import pandas as pd

# Convert dates, allowing pandas to infer each date format individually
data['publish date'] = pd.to_datetime(data['publish date'], format='mixed')

# Continue with extracting the year and counting documents per year
data['Year'] = data['publish date'].dt.year
documents_per_year = data['Year'].value_counts().sort_index()

first_date = data['publish date'].min()
last_date = data['publish date'].max()

print(f'Start: {first_date}, End: {last_date}')
print(documents_per_year)


Start: 2001-01-16 20:24:00, End: 2026-07-03 00:00:00
Year
2001      1
2014      7
2015     20
2016     73
2017    286
2018    256
2019    253
2020    259
2021    141
2022    356
2023    503
2024    375
2026      2
Name: count, dtype: int64


## Download PDFs using the Links

In [4]:
# Directory to save downloaded PDFs
download_directory = r"D:\Swapnil's personal space\Research\System And Contextual information integration- Scii\Final Tender Code\Data collection\Step-2 Doc -collection with Filtered Links\2.2 Data filteration\Downloaded_PDFs"


# Create the download directory if it does not exist
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Configure Selenium options
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--window-size=1920,1080')

# Set preferences to auto-download PDFs
prefs = {
    "download.default_directory": download_directory,  # Set download directory
    "plugins.always_open_pdf_externally": True  # Auto-download PDFs without prompting
}
chrome_options.add_experimental_option("prefs", prefs)

# Initialize the WebDriver using webdriver-manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def is_download_complete(download_dir):
    """
    Check if there are any incomplete downloads in the download directory.
    """
    for filename in os.listdir(download_dir):
        if filename.endswith('.crdownload'):  # Chrome's extension for in-progress downloads
            return False
    return True

def wait_for_download_to_complete(download_dir, timeout=60):
    """
    Wait until all downloads are complete or the timeout is reached.
    """
    start_time = time.time()
    while True:
        if is_download_complete(download_dir):
            print("Download completed.")
            return True
        elif time.time() - start_time > timeout:
            print("Download timed out.")
            return False
        time.sleep(1)  # Wait for 1 second before checking again

def download_pdf_with_selenium(url, index):
    """
    Use Selenium to open the URL and trigger the PDF download.
    """
    try:
        driver.get(url)
        
        # Wait for the page to load completely
        time.sleep(0.5)  # Increased initial delay to ensure the PDF viewer starts loading
        
        # Wait for download to complete
        if wait_for_download_to_complete(download_directory):
            print(f"Download finished for: {url}")
        else:
            print(f"Download not completed within the timeout for: {url}")

    except Exception as e:
        print(f"Failed to download PDF from {url} - Error: {e}")
        # Save the page source for debugging
        with open(f"failed_page_{index + 1}.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)

def main():
    for index, row in data.iterrows():
        url = row['document link']
        print(f"Processing PDF {index + 1}/{len(data)}: {url}")

        download_pdf_with_selenium(url, index)

    # Quit the driver after processing all URLs
    driver.quit()

if __name__ == "__main__":
    main()



Processing PDF 1/2532: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/NOC_1223.pdf
Download completed.
Download finished for: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/NOC_1223.pdf
Processing PDF 2/2532: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/NOC_for_web_challange.pdf
Download completed.
Download finished for: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/NOC_for_web_challange.pdf
Processing PDF 3/2532: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/Noc_03052024162706.pdf
Download completed.
Download finished for: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/Noc_03052024162706.pdf
Processing PDF 4/2532: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/Noc_RES_962.PDF
Download completed.
Download finished for: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/manualTender/Noc_RES_962.PDF
Processing PDF 5/2532: https://aiimsbhopal.edu.in/AIIMSFiles/procurement/man

In [5]:
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_colwidth', None) 